<a href="https://colab.research.google.com/github/player1537/Train-Bloom-560m/blob/main/Create_Wizard_Vicuna_Unfiltered_Summarized_70k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%script bash
TMPDIR=${TMPDIR:-${TMP:-/tmp}}
TMPOUT=${TMPDIR:?}/pip.text
INSTALL=(
  transformers
  torch
  datasets
  tqdm
  accelerate
  peft
  huggingface_hub
)

if ! pip install --upgrade "${INSTALL[@]}" &>"${TMPOUT:?}"; then
  cat "${TMPOUT:?}" >&2
  exit 1
fi

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
DO_EXPLODE = (
    # True
    False
)
DO_SUMMARIZE = (
    # True
    False
)
DO_IMPLODE = (
    True
    # False
)

REPOSITORY = 'player1537/wizard-vicuna-unfiltered-summarized-70k'
EXPLODE_BRANCH = 'explode'
SUMMARIZE_BRANCH = 'summarize'
SUMMARIZE_SPLITS = [
  "1of1of1of10",
  "1of1of2of10",
  "1of1of3of10",
  "1of1of4of10",
  "1of1of5of10",
  "1of1of6of10",
  "1of1of7of10",
  "1of1of8of10",
  "1of3of9of10",
  "2of3of9of10",
  "3of3of9of10",
  "1of5of10of10",
  "2of5of10of10",
  "3of5of10of10",
  "4of5of10of10",
  "5of5of10of10",
]
IMPLODE_BRANCH = 'main'
IMPLODE_SPLIT = 'train'

DEVICE = (
  'cpu'
  # 'cuda'
)
MODEL = (
  # 't5-large'
  # 'husseinMoh/t5-small-finetuned-text-simplification'
  't5-small'
)
DTYPE = (
  # 'float32'
  'float16'
)
TOKENIZER = (
  MODEL
  # 't5-large'
  # 'husseinMoh/t5-small-finetuned-text-simplification'
)
TASK = (
  'summarization'
)
DATASET = (
  'ehartford/wizard_vicuna_70k_unfiltered'
)
BATCH_SIZE = (
  128
  # 64
  # 32
  # 16
)
SHARDS = 10
SHARD = 9
MINISHARDS = 5
MINISHARD = 4
ROUNDING_FACTOR = (
  # 1
  # 2
  # 4
  8
)

In [ ]:
from tqdm.notebook import tqdm

def run(func=None, /):
  def wrapper(func, /):
    return func()

  if func is None:
    return wrapper
  else:
    return func()

In [ ]:
@run
def Model():
  from transformers import T5ForConditionalGeneration as Model
  Model = Model.from_pretrained
  return Model

In [ ]:
@run
def Dtype():
  import torch
  return {
      'float16': torch.float16,
      'float32': torch.float32,
  }.get

In [ ]:
@run
def Tokenizer():
  from transformers import AutoTokenizer as Tokenizer
  Tokenizer = Tokenizer.from_pretrained
  return Tokenizer

In [ ]:
@run
def Pipeline():
  from transformers import pipeline as Pipeline
  return Pipeline

In [ ]:
@run
def Dataset():
  from datasets import load_dataset as Dataset
  return Dataset

In [ ]:
if DO_EXPLODE or DO_SUMMARIZE:
  dtype = Dtype(DTYPE)

In [ ]:
if DO_EXPLODE or DO_SUMMARIZE:
  model = Model(MODEL, torch_dtype=dtype)
  print(model.get_memory_footprint())
# model

In [ ]:
if DO_EXPLODE or DO_SUMMARIZE:
  tokenizer = Tokenizer(TOKENIZER)
# tokenizer

In [ ]:
if DO_EXPLODE or DO_SUMMARIZE:
  pipeline = Pipeline(
      task=TASK,
      model=model,
      tokenizer=tokenizer,
      device=DEVICE,
  )
# pipeline

In [ ]:
if DO_EXPLODE:
  dataset = Dataset(DATASET)
  dataset = dataset['train']
  dataset

In [ ]:
def explode(x):
  ret = {}

  for id, conversation in zip(x['id'], x['conversations']):
    for i, message in enumerate(conversation):
      ret.setdefault('id', []).append( id )
      ret.setdefault('index', []).append( i )
      ret.setdefault('from', []).append( message['from'] )
      ret.setdefault('value', []).append( message['value'] )

  return ret
if DO_EXPLODE:
  dataset = dataset.map(explode, batched=True, remove_columns=['id', 'conversations'])
  dataset

In [ ]:
def each(x):
  return pipeline.preprocess(x['value'])
if DO_EXPLODE:
  dataset = dataset.map(each, batched=True)
  dataset

In [ ]:
def each(x):
  try:
    n = x['input_ids'].index(0)
  except ValueError:
    n = len(x['input_ids'])

  return { 'n': n }
if DO_EXPLODE:
  dataset = dataset.map(each)
  dataset

In [ ]:
if DO_EXPLODE:
  dataset = dataset.sort('n')
  dataset

In [ ]:
if DO_EXPLODE:
  dataset.push_to_hub(
    REPOSITORY,
    branch=EXPLODE_BRANCH,
  )
  print(dataset)

elif DO_SUMMARIZE:
  dataset = Dataset(
    REPOSITORY,
    revision=EXPLODE_BRANCH,
    split='train',
  )
  print(dataset)

In [ ]:
if DO_SUMMARIZE:
  dataset = dataset.shard(SHARDS, SHARD, contiguous=True)
  print(dataset)

In [ ]:
if DO_SUMMARIZE:
  dataset = dataset.shard(MINISHARDS, MINISHARD, contiguous=True)
  print(dataset)

In [ ]:
def each(x):
  n = x['n']
  n = min(512, n)
  input_ids = x['input_ids'][:n]
  attention_mask = x['attention_mask'][:n]
  value = tokenizer.decode(input_ids)
  value = value.removeprefix('summarize: ')
  return {
      'n': n,
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'value': value,
  }

if DO_SUMMARIZE:
  dataset = dataset.map(each)
  print(dataset)

In [ ]:
%%time
def summarize(_):
  global SUMMARIZE_ERROR_OCCURRED

  from transformers.pipelines.pt_utils import KeyDataset
  import itertools
  from datasets import IterableDataset

  ret = { 'summary': [] }
  it = dataset
  it = sorted(dataset, key=lambda x: x['n'])
  # it = tqdm(dataset)
  it = iter(it)
  it = itertools.groupby(it, key=lambda x: min(512, (x['n']//ROUNDING_FACTOR)*ROUNDING_FACTOR))
  it = [(K, [g for g in G]) for K, G in it]
  it = tqdm(it, desc=f'length {it[0][0]} to {it[-1][0]}')
  for n, it in it:
    total = len(it)
    it = KeyDataset(it, 'value')
    it = pipeline(it, min_length=1*n//2, max_length=2*n//3, batch_size=BATCH_SIZE)
    it = tqdm(it, desc=f'length {n}', total=total, leave=False)
    # it = list(it)
    for x in it:
      x = x[0]
      x = x['summary_text']
      ret['summary'].append(x)
  return ret

if DO_SUMMARIZE:
  dataset = dataset.map(summarize, batched=True, batch_size=None)
  print(dataset)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10.3 µs


In [ ]:
if DO_SUMMARIZE:
  dataset = dataset.sort(['id', 'index'])
  print(dataset)

In [ ]:
if DO_SUMMARIZE:
  dataset.push_to_hub(
      REPOSITORY,
      branch=SUMMARIZE_BRANCH,
      split=f'{1+MINISHARD}of{MINISHARDS}of{1+SHARD}of{SHARDS}',
  )
  dataset

elif DO_IMPLODE:
  from datasets import concatenate_datasets

  datasets = []
  for split in SUMMARIZE_SPLITS:
    print(f'Starting split: {split}')
    dataset = Dataset(
        REPOSITORY,
        revision=SUMMARIZE_BRANCH,
        # data_files={
        #   split: SUMMARIZE_FILES[split],
        # },
        split=split,
        # data_files=SUMMARIZE_FILES,
    )
    datasets.append(dataset)

  dataset = concatenate_datasets(datasets)
  del datasets

Starting split: 1of1of1of10


Extracting data files:   0%|          | 0/16 [00:00<?, ?it/s]

Generating 1of1of1of10 split: 0 examples [00:00, ? examples/s]

Generating 1of1of2of10 split: 0 examples [00:00, ? examples/s]

Generating 1of1of3of10 split: 0 examples [00:00, ? examples/s]

...

Starting split: 1of5of10of10


Starting split: 2of5of10of10


Starting split: 3of5of10of10


Starting split: 4of5of10of10


Starting split: 5of5of10of10


In [ ]:
if DO_IMPLODE:
  dataset = dataset.sort(['id', 'index'])
  print(dataset)

Dataset({
    features: ['id', 'index', 'from', 'value', 'input_ids', 'attention_mask', 'n', 'summary'],
    num_rows: 245212
})


In [ ]:
def implode(_):
  import itertools

  ret = { 'id': [], 'conversations': [] }
  it = tqdm(dataset, total=len(dataset))
  it = iter(it)
  it = itertools.groupby(it, key=lambda x: x['id'])
  for id, it in it:
    messages = []
    for x in it:
      messages.append({
        'from': x['from'],
        'value': x['summary'],
      })

    ret['id'].append(id)
    ret['conversations'].append(messages)

  return ret

if DO_IMPLODE:
  dataset = dataset.map(implode, batched=True, batch_size=None, remove_columns=dataset.column_names)
  print(dataset)

Map:   0%|          | 0/245212 [00:00<?, ? examples/s]

  0%|          | 0/245212 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'conversations'],
    num_rows: 34598
})


In [ ]:
if DO_IMPLODE:
  dataset.push_to_hub(
    REPOSITORY,
    branch=IMPLODE_BRANCH,
    split=IMPLODE_SPLIT,
  )

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]